# Extract data from output files
### Analyze the output from a single LBANN run
March 9, 2020 \
April 6, 2020 : to store files in order of epochs \
April 21, 2020: added jupyter widgets to compare pixel intensity plots \
May 8, 2020: using all images for a given batch \
May 29, 2020: Modified for new update of LBANN. File names of images changed, so new extraction code. Also added code for computing chi-squared. \
June 17, 2020: Removed train_inp, train_gen and val_inp to reduce memory overhead. From now on, the code only analyzes val_gen 
June 26, 2020: Added gathering of steps and new chi-square quantities.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import subprocess as sp
import os
import glob
import sys

import itertools
import time
from scipy import fftpack
# from ipywidgets import interact, interact_manual,fixed, SelectMultiple, IntText, IntSlider, FloatSlider,SelectionSlider,BoundedIntText
from ipywidgets import *

In [2]:
%matplotlib widget

In [3]:
sys.path.append('/global/u1/v/vpa/project/jpt_notebooks/Cosmology/Cosmo_GAN/repositories/lbann_cosmogan/3_analysis')
from modules_image_analysis import *

[NbConvertApp] Converting notebook modules_image_analysis.ipynb to script
[NbConvertApp] Writing 17167 bytes to modules_image_analysis.py


In [4]:
### Transformation functions for image pixel values
def f_transform(x):
    return 2.*x/(x + 4. + 1e-8) - 1.

def f_invtransform(s):
    return 4.*(1. + s)/(1. - s + 1e-8)

In [5]:
# ### Other transformatino functinos
# ### Transformation functions for image pixel values

# def f_transform_new(x):
#     if x<=50:
#         a=0.03; b=-1.0
#         return a*x+b
#     elif x>50: 
#         a=0.5/np.log(300)
#         b=0.5-a*np.log(50)
#         return a*np.log(x)+b

# def f_invtransform_new(y):
#     if y<=0.5:
#         a=0.03;b=-1.0
#         return (y-b)/a
#     elif y>0.5: 
#         a=0.5/np.log(300)
#         b=0.5-a*np.log(50)
#         return np.exp((y-b)/a)
    

# def f_transform(x):
#     return np.vectorize(f_transform_new)(x)

# def f_invtransform(s):
#     return np.vectorize(f_invtransform_new)(s)

# f_transform_new(2000)

### Modules for Extraction

In [6]:
def f_get_files_df_sorted():
    '''
    Module to create Dataframe with filenames for each epoch and step
    Sorts by step and epoch
    '''
    
    ## Get images files and .npy arrays for each image in dump_outs folder
    t1=time.time()
    files_dict={}
#     keys=['train_gen','train_input','val_gen','val_input']
#     file_strg_lst=['model0-training*-gen_img*-output0.npy','model0-training*-inp_img*-output0.npy','model0-validation*-gen_img*-output0.npy','model0-validation*-inp_img*-output0.npy']
#     file_strg_lst=['sgd.training*_gen_img*_output0.npy','sgd.training*_inp_img*_output0.npy','sgd.validation*_gen_img*_output0.npy','sgd.validation*_inp_img*_output0.npy']

    keys=['val_gen']
    file_strg_lst=['sgd.validation*_gen_img*_output0.npy']
    
    for key,file_strg in zip(keys,file_strg_lst):
        files_dict[key]=np.array(glob.glob(main_dir+file_strg))
        if files_dict[key].shape[0]>1000 : 
            print('Warning the number of files is very large. Possibility of memory overload')
    
    df_files=pd.DataFrame([])
    dict1={}
    t1=time.time()
    ### First get sorted Dataframe with file names
    for key in keys:
        files_arr=files_dict[key]  # Get array of files
        print(key,len(files_arr))
        for fname in files_arr:
            ### Extract the Epoch number and step number from the file name
            dict1['img_type']=key
            dict1['epoch']=np.int32(fname.split('epoch')[-1].split('.')[1])
            dict1['step']=np.int64(fname.split('step')[-1].split('.')[1].split('_')[0])
            dict1['fname']=fname
            
            df_files=df_files.append(dict1,ignore_index=True)
    ## Sort values
    df_files=df_files.sort_values(by=['img_type','epoch','step']).reset_index(drop=True)
    # df_files
    t2=time.time()
    print("Time for Sorting",t2-t1)
    
    return df_files


def f_filter_epoch(df_input,num_sliced=1):
    '''
    Get just equally spaced steps for each epoch
    '''
    print('Extracting %s steps of each epoch'%(num_sliced))
    df_output=pd.DataFrame([])
#     for key in ['train_gen','train_input','val_gen','val_input']: 
    for key in ['val_gen']: 
        ### For each type of images, get list of epochs
        df1=df_input[df_input.img_type==key]
        epochs=np.unique(df1.epoch.values).astype(int)

        for epoch in epochs:### Extract the last few steps in each epoch
            arr_step=df1[df1.epoch==epoch].step.values   ## Get all steps
            idxs=np.round(np.linspace(0,len(arr_step)-1,num_sliced).astype(int)) ## Get indices with equal spacing 
            df2=df1[df1.step.isin(arr_step[idxs])]        ## Get dataframe with these steps
            df_output=df_output.append(df2)  
    
    return df_output.reset_index(drop=True)

def f_get_images_df(df_files):
    '''
    Read dataframe with file names, read files and create new dataframe with images as numpy arrays
    Also computes number of images with intensity beyond a cutoff
    '''
    
    def f_row(df_row):
        '''
        Extract image
        '''
        fname,key=df_row.fname,df_row.img_type
        a1=np.load(fname)
        if key.endswith('input'): 
            size=np.int(np.sqrt(a1.shape[-1])) ### Extract size of images (=128)
            batch_size=a1.shape[0] ### Number of batches
            samples=a1.reshape(batch_size,size,size)
        elif key.endswith('gen') : samples=a1[:,0,:,:]
        else : raise SystemError

        return samples
    
    def f_high_pixel(df_row,cutoff=0.9966):
        '''
        Get number of images with a pixel about max cut-off value
        '''
        max_arr=np.amax(df_row.images,axis=(1,2))
        num_large=max_arr[max_arr>cutoff].shape[0]

        return num_large
    
    t1=time.time()
    ##### Create new Dataframe with sorted images
    df=df_files.copy()
    df['images']=df.apply(lambda row: f_row(row), axis=1)
    t2=time.time()
    print("Time for Reading images",t2-t1)
    
    ### Store the number of images with large pixel value
    cutoff=0.9966
    df['num_large']=df.apply(lambda row: f_high_pixel(row,cutoff), axis=1)
    
    return df
    


In [7]:
def f_get_sample_epochs(df,img_type,start_epoch=None,end_epoch=None):
    '''
    Module to extract images for a range of epochs given a dataframe
    '''
    if start_epoch==None and end_epoch==None:
        max_epoch=np.int(np.max(df.epoch.values))
#         print(max_epoch)
        start_epoch=0; end_epoch=max_epoch
#     if end_epoch==None: end_epoch=start_epoch+1
    
    arr=df[(df.epoch>=start_epoch) & (df.epoch<=end_epoch) & (df.img_type==img_type)].images.values
    arr=np.vstack(arr)
    
    return arr


def f_get_step(df,img_type,epoch,step):
    '''
    Module to extract images for a specific step and epoch
    '''
    
    arr=df[(df.epoch==epoch) & (df.step==step) & (df.img_type==img_type)].images.values
    arr=np.vstack(arr)
    
    return arr

def f_get_step_group(df,img_type,step_list):
    '''
    Module to extract images for a range of epochs given a dataframe
    '''
    arr=df[(df.step.isin(step_list)) & (df.img_type==img_type)].images.values
    arr=np.vstack(arr)
    
    return arr

## Extract image data 

In [8]:
# fldr_name='20200529_111342_seed3273_80epochs'
# fldr_name='20200626_075510_batchsize_256/'
# fldr_name='20200626_174615_batchsize_512/'
fldr_name='20200629_090438_batchsize_512/'
main_dir='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_data/{0}/dump_outs/trainer0/model0/'.format(fldr_name)
print(main_dir)

/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_data/20200629_090438_batchsize_512//dump_outs/trainer0/model0/


In [9]:
### Extract validation data
fname='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/dataset_2_smoothing_200k/norm_1_train_val.npy'
s_val=np.load(fname,mmap_mode='r')[:8000][:,0,:,:]
print(s_val.shape)

(8000, 128, 128)


In [10]:
### Get dataframe with file names, sorted by epoch and step
df_files=f_get_files_df_sorted()
### Slice out rows to keep only the last few steps for each epoch.
df_files=f_filter_epoch(df_files,num_sliced=6)

#############################################################
### Read images one by one into a numpy array and create a new DataFrame
df_full=f_get_images_df(df_files)
print(df_full.shape)
# ### Filter to keep just one step per epoch
# df_full=f_filter_epoch(df_full,1)

val_gen 715
Time for Sorting 2.7421348094940186
Extracting 6 steps of each epoch
Time for Reading images 22.979392528533936
(300, 6)


In [27]:
# df_files.head(20)

## Chi-square

In [12]:
def f_compute_chisqr(df,s_input):
    ''' Compute chi-sqr values of pixel intensity histogram and spectrum for each row
    Uses the module f_pixel_intensity to compute histograms and f_compute_spectrum for spectrum
    '''
    
    def f_chisqr(df_row,val_hist,val_err,val_spec,val_spec_err,bins,transform):
        ''' Compute chi-sqr of rows wrt to input data'''
        
        val_dr=val_hist.copy()
        val_dr[val_dr<=0.]=1.0    ### Avoiding division by zero for zero bins
        
        ### Get all images in a batch
        sample=df_row.images if not transform else f_invtransform(df_row.images)

        ### Compute pixel histogram for row   ### !!Both pixel histograms MUST have same bins and normalization!
        gen_hist,gen_err=f_pixel_intensity(sample,plot=False,normalize=True,bins=bins,mode='avg')
        spec,spec_err=f_compute_spectrum(sample,plot=False)

        ### Compute chi-sqr
        ### Used in keras code : np.sum(np.divide(np.power(valhist - samphist, 2.0), valhist))
        ###  chi_sqr :: sum((Obs-Val)^2/(Val))
        sq_diff=(gen_hist-val_hist)**2        
        chi_sqr_list=[]
        
        for count,(start,end) in enumerate(zip([0,22,38,0],[22,38,None,None])):  # 4 lists : small, medium, large pixel values and full 
            chi_sqr_list.append(np.sum(np.divide(sq_diff[start:end],val_dr[start:end])))
        
        idx=None  # Choosing the number of histograms to use. Eg : -5 to skip last 5 bins
        
        chi_sqr_list.append(np.sum(np.divide(sq_diff[:idx],1.0))) ## chi-sqr without denominator division
        chi_sqr_list.append(np.sum(gen_err[:idx])/np.sum(val_err[:idx])) ## measures total spread in histograms wrt to input data
        
        ### computing the spectral loss chi-square
        chi_sqr_list.append(np.sum((val_spec[:50]-spec[:50])**2/(spec[:50]**2)))
        
#         chi_sqr1=np.sum(np.divide(np.power(gen_hist[:idx] - val_hist[:idx], 2.0), val_dr[:idx]))
#         chi_sqr2=np.sum(np.divide(np.power(gen_hist[:idx] - val_hist[:idx], 2.0), 1.0))
#         chi_sqr3=np.sum(gen_err[:idx])/np.sum(val_err[:idx])  ## measures total spread in histograms wrt to input data
        
        return chi_sqr_list
    
    ########################
    ###### Code starts ########
    transform=False  # If true, it computes histogram in the orignal scale of pixels ie. 0-2000 
    
    ## Get bins for histograms
    bins=np.concatenate([np.array([-0.5]),np.arange(0.5,20.5,1),np.arange(20.5,100.5,5),np.arange(100.5,1000.5,50),np.array([2000])]) #bin edges to use
    if not transform: bins=f_transform(bins)   ### scale to (-1,1)
#     bins=100
#     print(bins)
    
    ### Get pixel histogram of all input data
    val_hist,val_err=f_pixel_intensity(s_input,plot=False,normalize=True,bins=bins,mode='avg')    
    ### Computing spectrum ###
    val_spec,val_spec_err=f_compute_spectrum(s_input,plot=False)
    del s_input


    ### Get chi-sqr for each row (step-epoch) for generated data
    chi_sqrs=df.apply(lambda row: f_chisqr(row,val_hist=val_hist,val_err=val_err,val_spec=val_spec,val_spec_err=val_spec_err,bins=bins,transform=transform), axis=1).values
    chi_vals=np.array(list(zip(*chi_sqrs)))  ## transposing list of list
        
    chi_sqr_keys=['chi_sqr1a','chi_sqr1b','chi_sqr1c','chi_sqr1d','chi_sqr2','chi_img_var','chi_spec']
    for key,chi_val in zip(chi_sqr_keys,chi_vals):
        df[key]=chi_val
    
    return df

def f_get_best_chisqr_models(df):
    
    chi_sqr_keys=['chi_sqr1a','chi_sqr1b','chi_sqr1c','chi_sqr1d','chi_sqr2','chi_img_var','chi_spec']
    q_dict=dict(df.quantile(q=0.15,axis=0)[chi_sqr_keys])
    print(q_dict)
    
    df_sliced=df.query('chi_sqr1d < {0} & chi_spec < {1}'.format(q_dict['chi_sqr1d'],q_dict['chi_spec']))[['epoch','step','img_type','num_large']+chi_sqr_keys]
    
    return df_sliced


In [13]:
t1=time.time()
# df1=f_compute_chisqr(df_full.loc[[0,1,2,3]],s_val) # Test on small df
df_full=f_compute_chisqr(df_full,s_val)
t2=time.time()
print("Time to compute chi-sqr",t2-t1)

Time to compute chi-sqr 375.5384786128998


In [14]:
df=df_full.copy()

In [21]:
df_sliced=f_get_best_chisqr_models(df_full)
print(df_sliced.shape)

{'chi_sqr1a': 0.03588829026648885, 'chi_sqr1b': 0.006592220897737583, 'chi_sqr1c': 0.006187053051378571, 'chi_sqr1d': 0.07143567507842313, 'chi_sqr2': 0.008153712444081182, 'chi_img_var': 3.331211063385375, 'chi_spec': 0.7088753409848016}
(21, 11)


In [16]:
# df_full.sort_values(by=['chi_sqr1d'])[['epoch','chi_sqr1a']].head(20)
# Get row with min chi-sqr
# df.loc[df.chi_sqr1.idxmin(axis=1)][['epoch','step','chi_sqr1','chi_sqr2','chi_sqr3','img_type']]
# chi_sqr_keys=['chi_sqr1a','chi_sqr1b','chi_sqr1c','chi_sqr1d','chi_sqr2','chi_img_var','chi_spec']
# df_full[chi_sqr_keys].describe()
# df_full[chi_sqr_keys]

In [22]:
df_sliced

,epoch,step,img_type,num_large,chi_sqr1a,chi_sqr1b,chi_sqr1c,chi_sqr1d,chi_sqr2,chi_img_var,chi_spec
112,18.0,1876.0,val_gen,0,0.007126,0.026790,0.011093,0.045009,0.001716,4.745215,0.703618
113,18.0,1897.0,val_gen,0,0.007375,0.020837,0.006688,0.034900,0.001844,4.719203,0.672125
175,29.0,2919.0,val_gen,0,0.058320,0.006911,0.005336,0.070567,0.049767,4.175418,0.654354
198,33.0,3304.0,val_gen,0,0.012148,0.004810,0.008936,0.025895,0.006274,4.655401,0.351116
199,33.0,3318.0,val_gen,0,0.013496,0.005326,0.001932,0.020754,0.005357,4.640286,0.370312
200,33.0,3339.0,val_gen,0,0.011663,0.005176,0.006868,0.023708,0.006300,4.672358,0.347492
201,33.0,3353.0,val_gen,0,0.010248,0.006909,0.003488,0.020645,0.007225,4.580767,0.347383
202,33.0,3374.0,val_gen,0,0.014486,0.003980,0.005090,0.023556,0.005711,4.649281,0.378036
203,33.0,3395.0,val_gen,0,0.011991,0.004649,0.003003,0.019643,0.006346,4.700131,0.367839
258,43.0,4305.0,val_gen,0,0.041710,0.004173,0.002920,0.048803,0.029968,4.800277,0.110878


In [20]:
### Plot chi-sqr values
df_sliced.plot(x="step", y=["chi_sqr1d", "chi_img_var", "chi_spec"],style='.',marker='*')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### High Pixel images

In [ ]:
### Plot number of high pixel images
plt.figure()
plt.plot(df[df.img_type=='val_gen'].epoch,df[df.img_type=='val_gen'].num_large,linestyle='',marker='*')
plt.xlabel('Steps in Epochs')
plt.ylabel('Number of large pixel images from a batch set of 128 images')

In [ ]:
df[(df.num_large>0) &(df.img_type=='val_gen')][['epoch','step','num_large']]

## Compare samples

In [23]:
def f_widget_compare(sample_names,sample_dict,Fig_type='pixel',rescale=True,log_scale=True,bins=25,mode='avg',normalize=True,bkgnd=[]):
    '''
    Module to make widget plots for pixel intensity or spectrum comparison for multiple sample sets
    '''
#     ### Crop out large pixel values
#     for key in sample_names:
#         print(sample_dict[key].shape)
#         sample_dict[key]=np.array([arr for arr in sample_dict[key] if np.max(arr)<=0.994])
#         print(sample_dict[key].shape)
    
    img_list=[sample_dict[key] for key in sample_names]
    label_list=list(sample_names)
    
    
    bins=np.concatenate([np.array([-0.5]),np.arange(0.5,20.5,1),np.arange(20.5,100.5,5),np.arange(100.5,1000.5,50),np.array([2000])]) #bin edges to use
    
    if rescale: 
        for count,img in enumerate(img_list):
            img_list[count]=f_invtransform(img)
        if len(bkgnd): bkgnd=f_invtransform(bkgnd)
#         hist_range=(0,2000)
    else:
        bins=f_transform(bins)
#         hist_range=(-1,0.996)
    assert Fig_type in ['pixel','spectrum'],"Invalid mode %s"%(mode)
    
    if Fig_type=='pixel':
#         f_compare_pixel_intensity(img_lst=img_list,label_lst=label_list,normalize=normalize,log_scale=log_scale, mode=mode,bins=bins,hist_range=hist_range)
        f_compare_pixel_intensity(img_lst=img_list,label_lst=label_list,normalize=normalize,log_scale=log_scale, mode=mode,bins=bins,hist_range=None,bkgnd_arr=bkgnd)

    elif Fig_type=='spectrum':
        f_compare_spectrum(img_lst=img_list,label_lst=label_list,log_scale=log_scale,bkgnd_arr=bkgnd)


### Compare different steps

In [24]:
# img_list,labels_list=f_get_sample_epochs(df,'val_gen',10)

img_list,labels_list=[],[]
for a,b in df_sliced.iterrows():
    epoch,step=int(b.epoch),int(b.step)
    img_list.append(f_get_step(df,'val_gen',epoch,step))
    labels_list.append('%s:%s'%(str(epoch),str(step)))

dict_samples=dict.fromkeys(labels_list)
for key,val in zip(labels_list,img_list): dict_samples[key]=val

dict_samples.keys()
# ### Compare with input
# # dict_samples['keras']=s_keras
# dict_samples['input']=s_val
bkgnd=[]
bkgnd=s_val
interact_manual(f_widget_compare,sample_dict=fixed(dict_samples),
                sample_names=SelectMultiple(options=dict_samples.keys()),
                Fig_type=ToggleButtons(options=['pixel','spectrum']),bins=IntText(value=50),mode=['avg','simple'],bkgnd=fixed(bkgnd))



interactive(children=(SelectMultiple(description='sample_names', options=('18:1876', '18:1897', '29:2919', '33…

<function __main__.f_widget_compare(sample_names, sample_dict, Fig_type='pixel', rescale=True, log_scale=True, bins=25, mode='avg', normalize=True, bkgnd=[])>

### Plot step groups in best epochs

In [25]:
print(np.unique(df_sliced.epoch.values))
step_list=df_sliced[df_sliced.epoch==26].step.values
print(step_list)

[18. 29. 33. 43. 46.]
[]


In [26]:
# img_list,labels_list=f_get_sample_epochs(df,'val_gen',10)

img_list,labels_list=[],[]
for epoch in np.unique(df_sliced.epoch.values).astype(int):
    step_list=df_sliced[df_sliced.epoch==epoch].step.values
    print(epoch,step_list)
    img_list.append(f_get_step_group(df,'val_gen',step_list))
    labels_list.append('%s'%(str(epoch)))

dict_samples=dict.fromkeys(labels_list)
for key,val in zip(labels_list,img_list): dict_samples[key]=val

dict_samples.keys()
# # ### Compare with input
# # # dict_samples['keras']=s_keras
# # dict_samples['input']=s_val
# bkgnd=[]
bkgnd=s_val
interact_manual(f_widget_compare,sample_dict=fixed(dict_samples),
                sample_names=SelectMultiple(options=dict_samples.keys()),
                Fig_type=ToggleButtons(options=['pixel','spectrum']),bins=IntText(value=50),mode=['avg','simple'],bkgnd=fixed(bkgnd))



18 [1876. 1897.]
29 [2919.]
33 [3304. 3318. 3339. 3353. 3374. 3395.]
43 [4305. 4319. 4340. 4354. 4375. 4396.]
46 [4606. 4620. 4641. 4655. 4676. 4697.]


interactive(children=(SelectMultiple(description='sample_names', options=('18', '29', '33', '43', '46'), value…

<function __main__.f_widget_compare(sample_names, sample_dict, Fig_type='pixel', rescale=True, log_scale=True, bins=25, mode='avg', normalize=True, bkgnd=[])>

In [ ]:
# img_lst=[f_invtransform(i) for i in img_list]
# bins=np.concatenate([np.array([-0.5]),np.arange(0.5,20.5,1),np.arange(20.5,100.5,5),np.arange(100.5,1000.5,50),np.array([2000])]) #bin edges to use
# # bins=200
# f_compare_pixel_intensity(img_list,labels_list,normalize=True,log_scale=True, mode='avg',bins=bins,hist_range=None)
# f_compare_spectrum(img_list,labels_list,log_scale=True)


## View image block

In [ ]:
def f_plot_grid(arr,cols=16,fig_size=(15,5)):
    ''' Plot a grid of images
    '''
    size=arr.shape[0]    
    rows=int(np.ceil(size/cols))
    print(rows,cols)
    
    fig,axarr=plt.subplots(rows,cols,figsize=fig_size, gridspec_kw = {'wspace':0, 'hspace':0})
    if rows==1: axarr=np.reshape(axarr,(rows,cols))
    if cols==1: axarr=np.reshape(axarr,(rows,cols))
    
    for i in range(min(rows*cols,size)):
        row,col=int(i/cols),i%cols
        try: 
            axarr[row,col].imshow(arr[i],origin='lower',interpolation='nearest',cmap='cool', extent = [0, 128, 0, 128])
        # Drop axis label
        except Exception as e:
            print('Exception:',e)
            pass
        temp=plt.setp([a.get_xticklabels() for a in axarr[:-1,:].flatten()], visible=False)
        temp=plt.setp([a.get_yticklabels() for a in axarr[:,1:].flatten()], visible=False)
    
#     fig.subplots_adjust(wspace=0.00,hspace=0.000)
#     fig.tight_layout()

# img_arr=f_get_sample_epochs(df,'val_gen',46,48)[20:50,:,:]
f_plot_grid(img_arr,cols=6,fig_size=(10,5))


In [46]:
fname='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_data/20200629_090438_batchsize_512/dump_outs/trainer0/model0/sgd.validation.epoch.43.step.4319_gen_img_instance1_activation_output0.npy'
fname='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_data/generate_images/20200629_145233_gen_img_exagan/dump_outs/trainer0/model0/sgd.testing.epoch.0.step.0_gen_img_instance1_activation_output0.npy'
s_new=np.load(fname)[:,0,:,:]
print(s_new.shape)

(512, 128, 128)


In [47]:
f_plot_grid(s_new[100:118],cols=6,fig_size=(10,5))

3 6


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [45]:
f_plot_grid(s_new[100:118],cols=6,fig_size=(10,5))

3 6


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …